# [muenchen.digital](https://muenchen.digital) Adventskalender 2024 - Coding Challenge 2

Über den folgenden Button gelangst du zu einer interaktiven Version dieses Notebooks, in der du die Challenge selbst lösen kannst: 

[![Binder starten](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/it-at-m/advent-coding-challenges-24/HEAD?urlpath=lab%2Ftree%2Fchallenges%2Fchallenge2.ipynb)

## Datensatz

Unser Coding-Adventskalender dreht sich dieses Jahr komplett um Weihnachtsmärkte.

Genauer gesagt, um die Weihnachtsmärkte in München, denn für die gibt es seit diesem Jahr einen frei verfügbaren Datensatz.

Diesen Datensatz wollen wir in den 4 Challenges nutzen, um nach und nach mehr Informationen zu den Weihnachtsmärkten herauszufinden.

Du findest ihn unter folgendem Link: https://opendata.muenchen.de/dataset/weihnachtsmaerkte-muenchen/resource/312260ca-7346-4b85-a357-3764bcadc6dd

Schau dir den Datensatz gerne erst einmal im Browser an, um ihn kennenzulernen.

# Download des Datensatzes

**Der Datensatz wurde inzwischen noch einmal bezüglich der Koordinaten korrigiert, bitte geh sicher dass du die aktuellste Version (v5) verwendest.**

Du kannst den Datensatz entweder [manuell herunterladen](https://opendata.muenchen.de/dataset/92515550-b236-4a08-9259-260513889548/resource/312260ca-7346-4b85-a357-3764bcadc6dd/download/weihnachtsmarkte-muenchen-2024-v5.csv) und wieder in das Notebook hochladen, oder du kannst den folgenden Code-Block ausführen, um den Datensatz direkt in das Notebook zu laden.

In [5]:
import requests

url = "https://opendata.muenchen.de/dataset/92515550-b236-4a08-9259-260513889548/resource/312260ca-7346-4b85-a357-3764bcadc6dd/download/weihnachtsmarkte-muenchen-2024-v5.csv"
response = requests.get(url)

with open("weihnachtsmarkte-muenchen-2024-v5.csv", "wb") as file:
    file.write(response.content)

## Vorbereitung

Zur Vorbereitung laden wir den Datensatz, dieses Mal aber in GeoPandas, um die Koordinaten direkt als Geometrie-Objekte zu haben.

In [38]:
# Pandas importieren
import pandas as pd
import geopandas as gpd

# Datensatz laden
df = pd.read_csv("../challenges/weihnachtsmarkte-muenchen-2024-v5.csv")

# GeoDataFrame erstellen
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitude, df.latitude), crs="EPSG:4326")

# GeoDataFrame plotten
gdf.explore()

# Aufgabenstellung

Diese Woche ist es das Ziel, den Weihnachtsmarkt zu finden, der am nähesten am IT-Referat liegt.

Dafür sind folgende Schritte nötig:

- Bestimme die Koordinaten des IT-Referats mithilfe eines Reverse Lookup in `GeoPy` und `Nomatim`
- Berechne die Distanz zwischen dem IT-Referat und jedem Weihnachtsmarkt
- Finde den Weihnachtsmarkt, der am nächsten am IT-Referat liegt, indem du nach der minimalen Distanz sortierst

## Hinweise

- Die Adresse des IT-Referats ist "Agnes-Pockels-Bogen 33, 80992 München".
- Die Auflösung des Namen zur Koodinate ist hier als Beispiel gegeben: https://geopy.readthedocs.io/en/stable/#module-geopy.geocoders
- Die Distanz kann für das ganze GeoDataFrame mit der Methode `gdf.distance` berechnet werden.
- Zur Berechnung muss die Location des IT-Referats noch in einen Point aus `shapely` umgewandelt werden: https://shapely.readthedocs.io/en/stable/manual.html#points
- Eventuell musst du die Koordinaten in ein anderes Koordinatensystem umrechnen, um die Distanz in Metern zu erhalten: https://geopandas.org/en/stable/docs/user_guide/projections.html#re-projecting
- Die berechnete Distanz zum IT-Rathaus kannst du einfach mit in deinem GeoDataFrame in einer neuen Spalte speichern, um dann nach dieser zu sortieren.
- Die Distanz wird in Meter angegeben, du kannst also um Kilometer zu erhalten durch 1000 teilen.

## Musterlösung

Nach Ende des Gewinnspiels findest du auch eine Musterlösung auf der Startseite des Repositorys verlinkt.

## Deine Lösung

Ab hier kannst du mit deiner Lösung beginnen.
Denk daran, die bereits geladene Tabelle `df` zu benutzen 😉

In [32]:
# Nomatim aus Geopy importieren
from geopy.geocoders import Nominatim

# Geocoder initialisieren
geocoder = Nominatim(user_agent="munich-christmas-markets")

# Adresse definieren
adress = "IT-Referat München"

# Koordinaten geocoden
location = geocoder.geocode(adress)

# Koordinaten anzeigen
print(f"Die Koordinaten von '{location.raw['name']}' sind: {location.latitude}, {location.longitude}")

Die Koordinaten von 'IT-Referat der Landeshauptstadt München - Gebäudeteil Qubes' sind: 48.1740293, 11.533678194445871


In [45]:
from shapely.geometry import Point

# Punkt für IT-Referat erstellen
it_referat_point = Point(location.longitude, location.latitude)

# GeoDataFrame in ein projiziertes CRS umwandeln
gdf_projected = gdf.to_crs(epsg=32633)
it_referat_point_projected = gpd.GeoSeries([it_referat_point], crs="EPSG:4326").to_crs(epsg=32633).iloc[0]

# Distanz mit Geopandas berechnen und runden
gdf_projected["distance"] = gdf_projected.distance(it_referat_point_projected)
gdf_projected["distance"] = gdf_projected["distance"] / 1000
gdf_projected["distance"] = gdf_projected["distance"].round(3)

# GeoDataFrame anzeigen
gdf_projected["distance"]

0      5.141
1      5.364
2      4.580
3      4.911
4      5.111
5      4.651
6      5.370
7      4.558
8      5.381
9      2.380
10     5.848
11     6.869
12     7.440
13     5.959
14     4.970
15     4.145
16     1.337
17     7.693
18     6.297
19     5.973
20     6.915
21    27.527
22     6.299
23     5.983
24     8.296
25     6.447
26     6.924
Name: distance, dtype: float64

In [56]:
# Ergebnis berechnen
closest_market = gdf_projected.loc[gdf_projected["distance"].idxmin()]

print(f"Der nächstgelegene Weihnachtsmarkt ist der {closest_market['Marktname']} am Ort {closest_market['Ort']} und ist {closest_market['distance']} km entfernt.")

# 5 nächstgelegene Weihnachtsmärkte anzeigen
gdf_projected.sort_values(by="distance", ascending=True).head(5)

Der nächstgelegene Weihnachtsmarkt ist der Märchenbazar im Olympiapark am Ort Olympiapark und ist 1.337 km entfernt.


,Marktname,Ort,Startdatum,Enddatum,Datum_Anmerkungen,Website,Beschreibung,Adresse,latitude,longitude,geometry,distance
16,Märchenbazar im Olympiapark,Olympiapark,28.11.2024,29.12.2024,NaN,https://www.maerchenbazar.de,Ein nostalgischer Markt mit Zirkuszelten und e...,"Spiridon-Louis-Ring 100, 80809 München",48.167645,11.548905,POINT (243415.994 5340694.539),1.337
9,Neuhauser Weihnachtsmarkt,Rotkreuzplatz,25.11.2024,23.12.2024,NaN,http://www.weihnachtsmarkt-muenchen-neuhausen.de,Ein familiärer Markt mit einer Vielzahl von ha...,"Rotkreuzplatz, 80634 München",48.152634,11.533209,POINT (242173.887 5339078.966),2.380
15,Schwabinger Weihnachtsmarkt,Münchner Freiheit,29.11.2024,24.12.2024,NaN,https://www.schwabingerweihnachtsmarkt.de,Ein kreativer Markt mit Kunstzelt und vielfält...,"Münchner Freiheit, 80802 München",48.162665,11.586725,POINT (246202.895 5340015.486),4.145
7,Tollwood – das Winterfestival,Theresienwiese,26.11.2024,31.12.2024,Markt bis 23.12.,https://www.tollwood.de,Ein alternatives Festival mit Bio-Essen und ku...,"Theresienwiese, 80339 München",48.134479,11.549667,POINT (243307.109 5337006.364),4.558
2,Eiszauber am Karlsplatz-Stachus,Karlsplatz-Stachus,22.11.2024,12.01.2025,NaN,https://www.muenchnereiszauber.de,Mit Bayerns größter mobiler Eisarena ist diese...,"Karlsplatz, 80335 München",48.139079,11.566200,POINT (244559.789 5337462.459),4.580
